<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/10_11_22_RQ3_politifact_dgl_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install jsonlines

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
from pathlib import Path
base_dir = Path("/gdrive/MyDrive/ResearchFND")
assert base_dir.exists()

## Data

In [4]:
import pandas as pd
import ast
import json

In [5]:
dataset_id = 'politifact'
text_embeddings = 'ft' # options: sbert, ft

In [6]:
df = pd.read_csv(base_dir/f'{dataset_id}_agg.csv')
df.head(2)

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0


In [9]:
with open(base_dir/'t2u.json') as f:
    t2u = json.load(f)

with open(base_dir/'users_info.json') as f:
    users_info = json.load(f)

In [10]:
df['tweets'] = df.tweets.map(ast.literal_eval)

In [11]:
users_tweeted = df.tweets.map(lambda x: [int(e['user_id']) for e in x])
users_retweeted = df.retweets.map(lambda x: [t2u[str(e)] for e in x if (str(e) in t2u)])

In [12]:
len(users_tweeted), sum(users_tweeted.map(len) > 0)

(894, 149)

## GNN

### Data

In [16]:
#%%capture
!pip install dgl wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dgl-0.9.1-cp37-cp37m-manylinux1_x86_64.whl (4.9 MB)
     |████████████████████████████████| 1.9 MB 5.8 MB/s 
     |████████████████████████████████| 281 kB 43.4 MB/s 
     |████████████████████████████████| 182 kB 42.0 MB/s 
     |████████████████████████████████| 162 kB 57.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 49.0 MB/s 
     |████████████████████████████████| 158 kB 58.5 MB/s 
     |████████████████████████████████| 157 kB 55.3 MB/s 
     |████████████████████████████████| 157 kB 48.3 MB/s 
     |████████████████████████████████| 157 kB 48.1 MB/s 
     |████████████████████████████████| 157 kB 49.3 MB/s 
     |████████████████████████████████| 157 kB 46.7 MB/s 
     |████████████████████████████████| 157 kB 57.7 MB/s 
     |████████████████████████████████| 157 kB 57.8 MB/s 
     |███████████████████

In [17]:
%env DGLBACKEND=pytorch

env: DGLBACKEND=pytorch


In [18]:
import os
import json
import jsonlines
import numpy as np
import torch
import dgl

import wandb
import IPython.display as ipd

In [19]:
u2i = {}

follow_src = []
follow_dst = []
with jsonlines.open(base_dir/"followers.jsonl") as reader:
    for line in reader:
        v = line["user_id"]
        if v not in u2i:
            u2i[v] = len(u2i)
        for u in line["followers"]:
            if u not in u2i:
                u2i[u] = len(u2i)
            follow_src.append(u2i[u])
            follow_dst.append(u2i[v])

In [20]:
with jsonlines.open(base_dir/"following.jsonl") as reader:
    for line in reader:
        u = line["user_id"]
        if u not in u2i:
            u2i[u] = len(u2i)
        for v in line["following"]:
            if v not in u2i:
                u2i[v] = len(u2i)
            follow_src.append(u2i[u])
            follow_dst.append(u2i[v])

In [21]:
for u, info in users_info.items():
    u = int(u)
    if u not in u2i:
        u2i[u] = len(u2i)
    for v in info['followers']:
        v = int(v)
        if v not in u2i:
            u2i[v] = len(u2i)
        follow_src.append(u2i[v])
        follow_dst.append(u2i[u])
    for v in info['friends']:
        v = int(v)
        if v not in u2i:
            u2i[v] = len(u2i)
        follow_src.append(u2i[u])
        follow_dst.append(u2i[v])

In [22]:
tweet_src = []
tweet_dst = []

for v, l in users_tweeted.iteritems():
    if not len(l):
        continue
    for u in l:
        u = int(u)
        if u in u2i:
            tweet_src.append(u2i[u])
            tweet_dst.append(v)

In [23]:
for v, l in users_retweeted.iteritems():
    if not len(l):
        continue
    
    for u in l:
        u = int(u)
        if u not in u2i:
            u2i[u] = len(u2i)
        tweet_src.append(u2i[u])
        tweet_dst.append(v)

In [24]:
article_bg_embs = np.load(base_dir/'politifact_pt_biggraph_article_embeddings_128.npy')
article_bg_ids = np.load(base_dir/'politifact_pt_biggraph_article_idx_128.npy')

user_bg_embs = np.load(base_dir/'politifact_pt_biggraph_user_embeddings_128.npy')
user_bg_ids = np.load(base_dir/'politifact_pt_biggraph_user_idx_128.npy')

In [25]:
text_embs = np.load(base_dir/f'{dataset_id}_{text_embeddings}_fulltext_embeddings.npy')
text_embs.shape

(894, 300)

In [26]:
article_embs = np.zeros((len(df), article_bg_embs.shape[1]))
article_embs.shape

(894, 128)

In [27]:
article_embs[article_bg_ids] = article_bg_embs

In [ ]:
# user_bg_ids

In [ ]:
# len(user_bg_ids), len(u2i)

In [28]:
u_ids = np.array([u2i[u] for u in user_bg_ids if (u in u2i)])
valid_user = np.array([x in u2i for x in user_bg_ids])

In [29]:
user_bg_embs[valid_user].shape

(82560, 128)

In [30]:
i = 0
for x in user_bg_ids:
    if x in u2i:
        i+=1
i

82560

In [31]:
assert len(set([type(k) for k in u2i.keys()]))

In [32]:
user_embs = np.zeros((len(u2i), user_bg_embs.shape[1]))
user_embs[u_ids] = user_bg_embs[valid_user]

In [33]:
follow_src = torch.tensor(follow_src)
follow_dst = torch.tensor(follow_dst)
tweet_src = torch.tensor(tweet_src)
tweet_dst = torch.tensor(tweet_dst)

graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'tweet', 'article'): (tweet_src, tweet_dst),
    ('article', 'tweeted-by', 'user'): (tweet_dst, tweet_src)})

graph.nodes['user'].data['feat'] = torch.tensor(user_embs).float()
graph.nodes['article'].data['feat'] = torch.cat([torch.tensor(text_embs), torch.tensor(article_embs).float()], axis=-1)
graph.nodes['article'].data['label'] = torch.tensor((df.label=="real").to_numpy()).long()

In [34]:
graph

Graph(num_nodes={'article': 894, 'user': 639764},
      num_edges={('article', 'tweeted-by', 'user'): 4587, ('user', 'follow', 'user'): 695148, ('user', 'followed-by', 'user'): 695148, ('user', 'tweet', 'article'): 4587},
      metagraph=[('article', 'user', 'tweeted-by'), ('user', 'user', 'follow'), ('user', 'user', 'followed-by'), ('user', 'article', 'tweet')])

In [35]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(shuffle=True, random_state=124)

In [36]:
labels = graph.ndata['label']['article']

train_idx, valid_idx = next(skf.split(labels, labels))

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
sampler = dgl.dataloading.NeighborSampler([10, 10])
train_loader = dgl.dataloading.DataLoader(
    graph,
    {'article':train_idx},
    sampler,
    device=device,
    batch_size=64,
    shuffle=True,
    drop_last=False,
    num_workers=0
)

In [39]:
eval_sampler = dgl.dataloading.NeighborSampler([-1, -1])
eval_loader = dgl.dataloading.DataLoader(
    graph,
    {'article':valid_idx},
    eval_sampler,
    device=device,
    batch_size=64,
    shuffle=False,
    drop_last=False,
    num_workers=0
)

In [40]:
batch = next(iter(train_loader))

/usr/local/lib/python3.7/dist-packages/dgl/dataloading/dataloader.py:859: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(f'Dataloader CPU affinity opt is not enabled, consider switching it on '


### Model

In [41]:
from collections import defaultdict
import torch.nn as nn
import torch.nn.functional as F

d_emb_dict = defaultdict(lambda: 64)

def flatten_dict(d):
    for k, v in d.items():
        d[k] = v.flatten(1)
    return d

In [42]:
in_proj = dgl.nn.HeteroLinear({k:v.shape[-1] for k, v in graph.ndata['feat'].items()}, 64)

with torch.no_grad():
    h = in_proj(graph.ndata['feat'])

In [43]:
class Residual(nn.Module):

    def __init__(self, conv):
        super().__init__()
        self.conv = conv

    def forward(self, graph, x):
        h = self.conv(graph, x)
        res = x[1]
        return h + res

In [44]:
conv = dgl.nn.HeteroGraphConv({rel:Residual(dgl.nn.GraphConv(64, 64, allow_zero_in_degree=True)) for rel in graph.etypes})

In [45]:
blocks = batch[-1]
block = blocks[0]
x = block.ndata['feat']

with torch.no_grad():
    h = in_proj(x)
    res = conv(block, h)

In [46]:
res['article'].shape, res['user'].shape

(torch.Size([64, 64]), torch.Size([39, 64]))

In [47]:
class Encoder(torch.nn.Module):

    def __init__(self, d_in, d_h, etypes, dropout=0.0):
        super().__init__()
        self.layers = nn.ModuleList([
            dgl.nn.HeteroGraphConv({
                rel : Residual(dgl.nn.GraphConv(d_in, d_h, allow_zero_in_degree=True)) for rel in etypes
            }),
            dgl.nn.HeteroGraphConv({
                rel : Residual(dgl.nn.GraphConv(d_h, d_h, allow_zero_in_degree=True)) for rel in etypes
            })
        ])

    def forward(self, blocks, x):
        
        for layer, block in zip(self.layers, blocks):
            x = layer(block, x)
        return x

In [48]:
class GNN(nn.Module):

    def __init__(self, g, d_h, tgt_ntype):
        super().__init__()
        self.tgt_ntype = tgt_ntype
        self.in_proj = dgl.nn.HeteroLinear({k:v.shape[-1] for k, v in g.ndata['feat'].items()}, d_h)
        self.encoder = Encoder(d_h, d_h, g.etypes)
        self.head = nn.Linear(d_h, 2)

    def forward(self, blocks, x):
        h = self.in_proj(x)
        h = self.encoder(blocks, h)
        return self.head(h[self.tgt_ntype])

    @torch.no_grad()
    def get_embeddings(self, graph, x):
        h = self.emb(x)
        h = self.encoder(graph, h)
        return h[self.tgt_ntype]

In [49]:
model = GNN(graph, 128, 'article')

with torch.no_grad():
    logits = model(blocks, x)

In [50]:
logits.shape

torch.Size([64, 2])

In [51]:
def accuracy(logits, labels):
    return (logits.argmax(-1) == labels).float().mean()

In [52]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(shuffle=True, random_state=124)

In [53]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

metrics = [accuracy_score, f1_score, precision_score, recall_score]
def get_name(score_func):
    return score_func.__name__.split("_")[0]

In [54]:
class AverageMeter:

    def __init__(self, store_vals=False, store_avgs=False):
        self.store_vals = store_vals
        self.store_avgs = store_avgs
        if store_vals: self.values = []
        if store_avgs: self.avgs = []
        self.tot, self.n = 0, 0

    def update(self, v, n=1):
        if self.store_vals: self.values.append(v)
        self.n += n
        self.tot += v*n

    @property
    def avg(self):
        if self.n == 0:
            return
        return self.tot / self.n

    def reset(self):
        if self.store_avgs and self.avg: self.avgs.append(self.avg)
        self.tot, self.n = 0, 0

In [55]:
def train(fold, train_idx, valid_idx, params):

    model = GNN(graph, 128, 'article')
    opt = torch.optim.Adam(model.parameters(), params['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=10, verbose=True)

    train_loss = AverageMeter(store_avgs=True)
    train_acc = AverageMeter(store_avgs=True)
    valid_loss = AverageMeter(store_avgs=True)
    valid_acc = AverageMeter(store_avgs=True)

    best_acc = 0
    for epoch in range(params['n_epochs']):
        model.train()
        for batch in train_loader:
            blocks = batch[-1]
            x = blocks[0].ndata['feat']
            logits = model(blocks, x)
            
            labels = blocks[-1].dstdata['label']['article']
            loss = F.cross_entropy(logits, labels)
            acc = accuracy(logits, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()

            train_loss.update(loss.item(), len(labels))
            train_acc.update(acc, len(labels))

        model.eval()
        for i, batch in enumerate(eval_loader):
            blocks = batch[-1]
            x = blocks[0].ndata['feat']
            with torch.no_grad():
                logits = model(blocks, x)

                labels = blocks[-1].dstdata['label']['article']
                val_loss = F.cross_entropy(logits, labels)
                val_acc = accuracy(logits, labels)

                valid_loss.update(val_loss.item(), len(labels))
                valid_acc.update(val_acc, len(labels))
        
        scheduler.step(valid_loss.avg)
        wandb.log({'train_loss':loss.item(), 'train_acc':acc, 'valid_loss':val_loss.item(), 'valid_acc':val_acc}, step=epoch)
        print(f"{epoch+1:>3}: Train loss {train_loss.avg:.4f}, acc {train_acc.avg:.4f}%; validation loss {valid_loss.avg:.4f}, acc {valid_acc.avg:.4f}%")
        
        if valid_acc.avg >= best_acc:
            best_acc = valid_acc.avg
            torch.save(model.state_dict(), f'models/model-{fold}.pt')
        
        train_loss.reset()
        train_acc.reset()
        valid_loss.reset()
        valid_acc.reset()

        

    # load best model and evaluate
    model.load_state_dict(torch.load(f'models/model-{fold}.pt'))
    model.eval()
    preds = []
    targs = []
    for i, batch in enumerate(eval_loader):
        blocks = batch[-1]
        x = blocks[0].ndata['feat']
        with torch.no_grad():
            logits = model(blocks, x)

            labels = blocks[-1].dstdata['label']['article']
            
        preds.append(logits.argmax(-1).cpu().numpy())
        targs.append(labels.cpu().numpy())
    preds = np.concatenate(preds)
    targs = np.concatenate(targs)
    eval_results = {get_name(f):f(y_pred=preds, y_true=targs) for f in metrics}
    print("Final evaluation results:")
    for k,v in eval_results.items():
        print(f"{k:<16}{v:.4f}")
    
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=targs, preds=preds,
                            class_names=["Fake", "Real"])})

    return {
        'train_loss':train_loss,
        'train_acc':train_acc,
        'valid_loss':valid_loss,
        'valid_acc':valid_acc
    }

In [56]:
params = {
    "n_epochs":100,
    'bs': 16,
    'lr':1e-2,
    "seed":124,
}

labels = graph.ndata['label']['article']

In [57]:
if not os.path.exists('models'):
    os.mkdir('models')

GROUP = "sbert-fulltext-hetgnn-ua"
for fold_id, (train_idx, valid_idx) in enumerate(skf.split(labels, labels)):
    ipd.clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        log = train(fold_id, train_idx, valid_idx, params)
    break

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/dgl/dataloading/dataloader.py:859: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(f'Dataloader CPU affinity opt is not enabled, consider switching it on '
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  1: Train loss 0.5897, acc 0.6350%; validation loss 0.5152, acc 0.7821%
  2: Train loss 0.4540, acc 0.7874%; validation loss 0.4572, acc 0.7654%
  3: Train loss 0.3717, acc 0.8280%; validation loss 0.3686, acc 0.8492%
  4: Train loss 0.2835, acc 0.8937%; validation loss 0.3530, acc 0.8436%
  5: Train loss 0.2470, acc 0.9077%; validation loss 0.3263, acc 0.8883%
  6: Train loss 0.2383, acc 0.9147%; validation loss 0.4901, acc 0.7933%
  7: Train loss 0.2243, acc 0.9161%; validation loss 0.3557, acc 0.8436%
  8: Train loss 0.2100, acc 0.9231%; validation loss 0.3456, acc 0.8883%
  9: Train loss 0.1841, acc 0.9287%; validation loss 0.3654, acc 0.8492%
 10: Train loss 0.1827, acc 0.9301%; validation loss 0.3336, acc 0.8883%
 11: Train loss 0.1659, acc 0.9329%; validation loss 0.3463, acc 0.9106%
 12: Train loss 0.2006, acc 0.9175%; validation loss 0.3806, acc 0.8771%
 13: Train loss 0.1624, acc 0.9315%; validation loss 0.3836, acc 0.8827%
 14: Train loss 0.1977, acc 0.9119%; validation los

accuracy,▁
f1,▁
precision,▁
recall,▁
train_acc,▆▆█▁▆▆█▃▆███▆█▆█▆██▃████████▃███████████
train_loss,█▅▄█▃▅▁▄▇▃▂▂▃▁▃▁▃▂▁█▁▁▁▁▁▂▂▂▅▁▂▁▂▁▁▁▂▁▁▁
valid_acc,▆▅▂▅█▇▅▃█▃▃▁▂▂▁▂▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▃▃▃▃▃▃▃▃
valid_loss,▄▁▂▁▂▃▂▃▄▄▄▅▅▇▆▆▆▆▇▇▇▇▇█████████████████
accuracy,0.91061
f1,0.91304
precision,0.95455


In [58]:
model = GNN(graph, 128, "article")

model.load_state_dict(torch.load(f'models/model-{fold_id}.pt'))

<All keys matched successfully>